# 如何导出自己的原创微博

## 来由
号给炸了，收藏和微博全在里面，能看，但是没法操作。WTF   

## 环境python3
* jupyter notebook 
* selenium 这是一个自动化工具。就是可以自动模拟用户对微博的翻页、点击、滚动网页等动作
* lxml 这是网页的解析工具。我们获取到网页之后需要通过lxml去获取网页中的链接或者文本信息。 selenium库中也有自己的网页html解析防范，但是试用过还是lxml更强大一些

安装两个库

In [4]:
!pip install notebook
!pip install selenium
!pip install lxml

##  使用selenium登陆炸号微博

1. 从网络上下载chromedriver.exe
2. 修改下面chrome_driver_path变量的路径为本地chromedirver.exe的路径
3. 运行下面代码会打开chrome和微博首页
4. 输入自己的微博完成人工登陆

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from lxml import etree
#coding:utf-8
from lxml import html 
from tqdm import tqdm
 
chrome_driver_path = r'C:\Users\xxxxx\chromedriver_win32 (1)\chromedriver.exe'
driver = webdriver.Chrome(executable_path=r"C:\Users\ESS User\Desktop\BOOK\chromedriver_win32 (1)\chromedriver.exe")
driver.get("http://www.weibo.com")

## 设置一些变量。
1. 默认保存的原创微博放在mytweets下面
2. 查看自己的微博有多少页，比如打开自己的微博，滚到底，滚到不加载新页面的时候，就出现下一页和页数信息了。
3. 确认自己开始导出收藏的页数，比如第3页到第30页。那page_start = 3, page_end = 30就好

In [6]:
save_my_tweets = './mytweets.txt'
page_start = 1
page_end = 120

## 运行下面代码，开始微博下载

In [ ]:
def scroll_down_end(driver):
    '''
    参考stackoverflow 如何滚动到页面底部
    '''
    start = time.time()
    while True and (time.time()-start) <10:
    
        SCROLL_PAUSE_TIME = 0.5
        
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
        
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
   
def get_better_html_tree(driver):  
    mark_html_path = 'heyhey'
    with  open(mark_html_path,'wb') as f:
        f.write(driver.page_source.encode('utf-8')) 
    tree = html.parse(mark_html_path)
    return tree     



for pg_num in range(page_start, page_end+1):

    urllink = f'https://www.weibo.com/1826134871/profile?is_search=0&visible=0&is_all=1&is_tag=0&profile_ftype=1&page={pg_num}#feedtop'
    driver.get(urllink)
    scroll_down_end(driver)
    
    tree = get_better_html_tree(driver)
    mytweets = tree.xpath("//div[@tbinfo and not(@minfo)]")
     
    for t in mytweets:
        tweet_content = t.xpath(".//div[@class='WB_text W_f14']")[0].text_content().strip()
        # 处理一些文本编码问题
        tweet_content = tweet_content.replace(u'\u200b', u' ').replace('\xa0','').replace('\U0001f449','').replace('\xa1', '').replace('\ue627','')
        timediv = t.xpath(".//div[@class='WB_from S_txt2']")[0]
        time_tok = timediv.text_content().strip()[:16]
        with open(save_my_tweets, 'a') as f:
            f.write(time_tok + '\n')
            f.write(tweet_content + '\n')
            print(time_tok)
            print(tweet_content)

## 导出结果的txt

按照时间和微博内容进行保存
```
2019-7-7 00:24 
天地有正气，杂然赋流形。
2019-7-10 00:24 
下则为河岳，上则为日星
```


## 思路
主要难点在于如何分析是不是自己的微博。通过inspect发现，div中有tbinfo属性，无minfo的才是原创微博。如果有minfo属性，则是转发微博
比如下面，上面的div是原创微博，下面的是转发微博，可以在自己的微博里验证
![title](weibo_html.png)


## Rebuttal 思考
1. 有些原创微博很长，需要点击“完全展开”才可以全部来看，不然只能看到部分内容。 可以写一个这种链接点击操作。目前没有实现。
2. 对于每一页的微博打开，是打开这个页面，然后不停滚到十秒钟，假定滚动到底了。然后再提取网页。通过观察大部分情况10秒钟就滚到底了